In [1]:
from langchain_experimental.chat_models.llm_wrapper import Llama2Chat

from gba.client import LlamaCppClient
from gba.client.chat import MistralInstruct

# Proxy for 8-bit quantized Mistral-7B-Instruct-v0.2
mistral_instruct = MistralInstruct(
    llm=LlamaCppClient(url="http://localhost:8081/completion", temperature=-1),
)

# Proxy for 8-bit finetuned Mistral-7B-v0.1 planner
mistral_finetuned = MistralInstruct(
    llm=LlamaCppClient(url="http://localhost:8082/completion", temperature=-1),
)

# Proxy for 4-bit quantized CodeLlama-7B-Instruct
code_llama = Llama2Chat(
    llm=LlamaCppClient(url="http://localhost:8088/completion", temperature=-1),
)

# Proxy for 8-bit quantized NexusRaven-V2-13B
nexus_raven = LlamaCppClient(url="http://localhost:8089/completion", temperature=-1)

In [3]:
from gba.agent import Agent
from gba.client import ChatClient
from gba.planner import FineTunedPlanner
from gba.search import SearchEngine
from gba.store import DocumentStore
from gba.summary import ResultSummarizer
from gba.tools import *

from example_docs import DOCUMENTS
from example_funcs import create_event

store = DocumentStore(path=".chroma")
engine = SearchEngine(store=store)

if store.count() == 0:
    for i, document in enumerate(DOCUMENTS):
        store.add(identifier=str(i), document=document)

summarizer = ResultSummarizer(model=mistral_instruct)

tools = [
    AskTool(),
    CalculateTool(model=code_llama, summarizer=summarizer),
    FunctionCallTool(model=nexus_raven, fn=create_event),
    FunctionCallTool(model=nexus_raven, fn=engine.search_images),
    FunctionCallTool(model=nexus_raven, fn=engine.search_internet, summarizer=summarizer),
    RespondTool(model=mistral_instruct),
]

client = ChatClient(model=mistral_finetuned)
planner = FineTunedPlanner(client=client)
agent = Agent(planner=planner, tools=tools)

In [4]:
agent.run("what is Leo DiCaprio's current girlfriend's age raised to the 0.24 power?")

Task: Search for Leo DiCaprio's current girlfriend's name and age.
Call: search_internet(query='Leo DiCaprio')
Observation: Vittoria Ceretti is Leo DiCaprio's current girlfriend.

Task: Search for Vittoria Ceretti's current age.
Call: search_internet(query='Vittoria Ceretti')
Observation: Vittoria Ceretti is currently 25 years old.

Task: Calculate the value of 25 raised to the power of 0.24.
```python
result = 25 ** 0.24
```
Observation: The value of 25 raised to the power of 0.24 is approximately 2.16524.



'The value of 25 raised to the power of 0.24 is approximately 2.16524.'

In [5]:
agent.run("How many years did octopuses appear earlier "
          "on Earth, compared to elephants?")

Task: Search Wikipedia for the first appearance of octopuses on Earth.
Call: search_internet(query='first appearance of octopuses on Earth')
Observation: Octopuses first appeared on Earth during the Cretaceous period, around 155 million years ago.

Task: Search Wikipedia for the first appearance of elephants on Earth.
Call: search_internet(query='first appearance of elephants on Earth')
Observation: Elephants first appeared on Earth around 55 million years ago.

Task: Calculate the difference in years between 155 million and 55 million.
```python
result = 155 - 55
```
Observation: The difference is 100 million years.



'Octopuses appeared approximately 100 million years earlier than elephants on Earth.'

In [6]:
agent.run("Add Martin's birthday party, starting Dec. 17th 2023 8pm, "
          "to my calendar and respond with a one-line invitation")

Task: Create an event titled 'Martin's Birthday Party' starting on December 17th at 8pm in 2023.
Call: create_event(title='Martin\'s Birthday Party', date='12/17/2023', time='8pm')
Observation: Event Martin's Birthday Party successfully added to calendar, date=12/17/2023, time=8pm



"You're invited to Martin's Birthday Party on December 17th, 2023 at 8pm."

In [7]:
agent.run("Hello, how are you?")

"I'm just a computer program, but I'm here to help answer any questions you might have. How about you? How has your day been?"